# Análisis Exploratorio de Datos

Hemos hablado mucho de datos, todo muy de libro o juguete. Esta clase intentará acercarte a algunos de los principales desafíos a la hora de trabajar con distintas fuentes de datos y los problemas usuales que podrías encontrar.

## Fuentes de datos

Para variar un poco, utilizaremos la librería `pathlib` en lugar de `os` para manejar directorios. El paradigma es un poco distinto, en lugar de muchas funciones, la filosofía es tratar a los directorios como objetos que tienen sus propios métodos y operaciones.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
data_path = Path().resolve().parent / "data"
print(data_path)

/home/jovyan/work/mat281_2020S2/data


### CSV

Del inglés _Comma-Separated Values_, los archivos CSV utilizan comas (",") para separar valores y cada registro consiste de una fila. 

- Pros:
    * Livianos.
    * De fácil entendimiento.
    * Editables usando un editor de texto.
- Contras:
    * No está totalmente estandarizado (e.g. ¿Qué pasa si un valor tiene comas?)
    * Son sensible al _encoding_ (es la forma en que se codifica un carácter).
    
Pandas posee su propia función para leer csv: `pd.read_csv()`.

In [3]:
# Documentación
# pd.read_csv?

Un ejemplo de _encoding_ incorrecto

In [4]:
pd.read_csv(data_path / "encoding_example.csv", sep=",", encoding="gbk")

,nombre,apellido,edad
0,Juan,P茅rez,12.0
1,Le贸n,Pardo,29.0
2,Jos茅,Nu帽ez,NaN


Mientras que el mismo dataset con el encoding correcto luce así

In [5]:
pd.read_csv(data_path / "encoding_example.csv", sep=",", encoding="utf-8")

,nombre,apellido,edad
0,Juan,Pérez,12.0
1,León,Pardo,29.0
2,José,Nuñez,NaN


### JSON

Acrónimo de _JavaScript Object Notation_, utilizado principalmente para intercambiar datos entre una aplicación web y un servidor.

- Pros:
    * Livianos.
    * De fácil entendimiento.
    * Editables usando un editor de texto.
    * Formato estandarizado.
- Contras:
    * La lectura con pandas puede ser un poco complicada.
    
Pandas posee su propia función para leer JSON: `pd.read_json()`.

In [6]:
# pd.read_json?

Se parecen mucho a los diccionarios de python pero en un archivo de texto.

In [7]:
!head ../data/json_example.json

{
  "integer": {
    "0": 5,
    "1": 5,
    "2": 9,
    "3": 6,
    "4": 6,
    "5": 9,
    "6": 7,
    "7": 1,


In [8]:
pd.read_json(data_path / "json_example.json", orient="columns").head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


### Pickle

Es un módulo que implementa protocolos binarios de serialización y des-serialización de objetos de Python.

* Pros
    - Puede representar una inmensa cantidad de tipos de objetos de python.
    - En un contexto de seguridad, como no es legible por el ser humano (representación binaria) puede ser útil para almacenar datos sensibles.
* Contras:
    - Solo Python.
    - Si viene de un tercero podría tener contenido malicioso.
    
Pandas posee su propia función para leer pickles: `pd.read_pickle()`.

In [9]:
# pd.read_pickle?

In [10]:
pd.read_pickle(data_path / 'nba.pkl').head()

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University


### SQL

Conocimos las bases de datos relacionales SQL en clases anteriores y como recordarás existe la función `pd.read_sql()`, lo interesante aquí es que debes crear una conexión antes de poder leer la base de datos. Cada Sistema de Gestión de Bases de Datos Relacionales (_Relational Database Management System_ o RDBMS) tiene su propia forma de conectarse.

In [11]:
# pd.read_sql?

In [12]:
import sqlite3
connector = sqlite3.connect(data_path / "chinook.db")
pd.read_sql_query("select * from albums", con=connector).head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


### API

¿Has escuchado el término __API__? Fuera de todo tecnicismo, las APIs (_Application Programming Interface_) permiten hacer uso de funciones ya existentes en otro software (o de la infraestructura ya existente en otras plataformas) para no estar reinventando la rueda constantemente, reutilizando así código que se sabe que está probado y que funciona correctamente. Por ejemplo, cuando haces una compra online y utilizas WebPay o una página utiliza los mapas de GoogleMaps. ¡Hay APIs en todos lados!

Utilizaremos la API de Open Notify para obtener cuántas personas hay en el espacio en este momento ([link](http://open-notify.org/Open-Notify-API/People-In-Space/)).

In [13]:
import requests

In [14]:
response = requests.get("http://api.open-notify.org/astros.json")
print(f"response has type {type(response)}")
print(response)

response has type <class 'requests.models.Response'>
<Response [200]>


Puedes acceder a su contenido como un JSON de la siguiente manera

In [15]:
response.json()

{'number': 3,
 'people': [{'craft': 'ISS', 'name': 'Chris Cassidy'},
  {'craft': 'ISS', 'name': 'Anatoly Ivanishin'},
  {'craft': 'ISS', 'name': 'Ivan Vagner'}],
 'message': 'success'}

Lo cual en la práctica lo carga como un diccionario en Python

In [16]:
type(response.json())

dict

Por lo que podemos cargar ciertas estructuras a dataframes con métodos de pandas que utilicen diccionarios. Por dar un ejemplo, dentro del JSON obtenido hay una lista de personas.

In [17]:
pd.DataFrame.from_dict(response.json()["people"])

,craft,name
0,ISS,Chris Cassidy
1,ISS,Anatoly Ivanishin
2,ISS,Ivan Vagner


## Manos a la obra

El análisis exploratorio de datos es una forma de analizar datos definido por John W. Tukey (E.D.A.: Exploratory data analysis) es el tratamiento estadístico al que se someten las muestras recogidas durante un proceso de investigación en cualquier campo científico. Para mayor rapidez y precisión, todo el proceso suele realizarse por medios informáticos, con aplicaciones específicas para el tratamiento estadístico. 

El análisis exploratorio de datos debería dar respuestas (al menos) a lo siguiente:
1. ¿Qué pregunta(s) estás tratando de resolver (o probar que estás equivocado)?
2. ¿Qué tipo de datos tiene y cómo trata los diferentes tipos?
3. ¿Qué falta en los datos y cómo los maneja?
4. ¿Qué hacer con los datos faltantes, outliers o información mal inputada?
5. ¿Se puede sacar más provecho a los datos ?


### Ejemplo: Datos de terremotos

El dataset `earthquakes.csv` contiene la información de los terremotos de los países durante el año 2000 al 2011. Debido a que la información de este dataset es relativamente fácil de trabajar, hemos creado un dataset denominado `earthquakes_contaminated.csv` que posee información contaminada en cada una de sus columnas. De esta forma se podrá ilustrar los distintos inconvenientes al realizar análisis exploratorio de datos.

In [18]:
pd.read_csv(data_path / "earthquakes.csv").head()

,Año,Pais,Magnitud
0,2011,Turkey,7.1
1,2011,India,6.9
2,2011,Japan,7.1
3,2011,Burma,6.8
4,2011,Japan,9.0


In [19]:
earthquakes = pd.read_csv(data_path / "earthquakes_contaminated.csv")
earthquakes.head()

,Año,Pais,Magnitud,Informacion
0,2000,Turkey,6,info no valiosa
1,2000,Turkmenistan,7,info no valiosa
2,2000,Azerbaijan,6.5,info no valiosa
3,2000,Azerbaijan,6.8,info no valiosa
4,2000,Papua New Guinea,8,info no valiosa


__Variables__

* __Pais__:
    - Descripción: País del devento sísmico.
    - Tipo: _string_
    - Observaciones: No deberían encontrarse nombres de ciudades, comunas, pueblos, estados, etc.
* Año:
    - Descripción: Año del devento sísmico.
    - Tipo: _integer_
    - Observaciones: Los años deben estar entre 2000 y 2011.
* Magnitud:
    - Descripción: Magnitud del devento sísmico medida en [Magnitud de Momento Sísmico](https://en.wikipedia.org/wiki/Moment_magnitude_scale).
    - Tipo: _float_
    - Observaciones: Magnitudes menores a 9.6.
* Informacion:
    - Descripción: Columna contaminante.
    - Tipo: _string_
    - Observaciones: A priori pareciera que no entrega información a los datos.

A pesar que la magnitud es un _float_, el conocimiento de los datos nos da información relevante, pues el terremoto con mayor magnitud registrado a la fecha fue el de Valdivia, Chile el 22 de mayo de 1960 con una magnitud entre 9.4 - 9.6. 

__Los datos son solo bytes en el disco duro si es que no entregan valor y conocimiento.__

#### ¿Qué pregunta(s) estás tratando de resolver (o probar que estás equivocado)?

A modo de ejemplo, consideremos que que queremos conocer la mayor magnitud de terremoto en cada país a lo largo de los años.

#### ¿Qué tipo de datos tiene y cómo trata los diferentes tipos?

Por el conocimiento de los datos sabemos que `Pais` e `Información` son variables categóricas, mientras que `Año` y `Magnitud` son variables numéricas.

Utilizemos las herramientas que nos entrega `pandas`.

In [20]:
earthquakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Año          226 non-null    object
 1   Pais         226 non-null    object
 2   Magnitud     225 non-null    object
 3   Informacion  220 non-null    object
dtypes: object(4)
memory usage: 7.2+ KB


In [21]:
earthquakes.describe(include="all").T

,count,unique,top,freq
Año,226,16,2003,31
Pais,226,74,Indonesia,27
Magnitud,225,45,6.4,14
Informacion,220,3,info valiosa,166


In [22]:
earthquakes.dtypes

Año            object
Pais           object
Magnitud       object
Informacion    object
dtype: object

Todas las columnas son de tipo `object`, sospechoso. Además, algunas no tienen datos.

__Tip__: Típicamente se utilizan nombres de columnas en minúsculas y sin espacios. Un truco es hacer lo siguiente:

In [23]:
earthquakes = earthquakes.rename(columns=lambda x: x.lower().strip())
earthquakes.head()

,año,pais,magnitud,informacion
0,2000,Turkey,6,info no valiosa
1,2000,Turkmenistan,7,info no valiosa
2,2000,Azerbaijan,6.5,info no valiosa
3,2000,Azerbaijan,6.8,info no valiosa
4,2000,Papua New Guinea,8,info no valiosa


Se le aplicó una función `lambda` a cada nombre de columna! Puum! 

#### ¿Qué falta en los datos y cómo los maneja?

No es necesario agregar más variables, pero si procesarla.

#### ¿Qué hacer con los datos faltantes, outliers o información mal inputada?

A continuación iremos explorando cada una de las columnas.

In [24]:
for col in earthquakes:
    print(f"La columna {col} posee los siguientes valores únicos:\n {earthquakes[col].sort_values().unique()}\n\n")

La columna año posee los siguientes valores únicos:
 ['1990' '1997' '1999' '2000' '2001' '2002' '2003' '2004' '2005' '2006'
 '2007' '2008' '2009' '2010' '2011' 'dos mil uno' nan]


La columna pais posee los siguientes valores únicos:
 ['Afghanistan' 'Afghanistan ' 'Algeria' 'Algeria ' 'Argentina'
 'Azerbaijan' 'Azerbaijan ' 'Bangladesh' 'Burma ' 'Chile' 'Chile ' 'China'
 'China ' 'Colombia' 'Costa Rica' 'Costa Rica '
 'Democratic Republic of the Congo' 'Democratic Republic of the Congo '
 'Dominican Republic' 'Ecuador' 'El Salvador ' 'Greece' 'Greece '
 'Guadeloupe' 'Guatemala' 'Haiti ' 'India' 'India ' 'Indonesia'
 'Indonesia ' 'Iran' 'Iran ' 'Iran, 2005 Qeshm earthquake' 'Italy'
 'Italy ' 'Japan' 'Japan ' 'Kazakhstan' 'Kyrgyzstan ' 'Martinique'
 'Mexico ' 'Morocco' 'Morocco ' 'Mozambique' 'New Zealand' 'New Zealand '
 'Nicaragua' 'Pakistan' 'Pakistan ' 'Panama' 'Papua New Guinea' 'Peru'
 'Peru ' 'Philippines' 'Russian Federation' 'Rwanda' 'Samoa ' 'Serbia'
 'Slovenia' 'Solomon Island

* En la columna `año` se presentan las siguientes anomalías:
    * Datos vacíos.
    * Años sin importancia: Se ha establecido que los años de estudios son desde el año 2000 al 2011.
    * Nombres mal escritos: en este caso sabemos que 'dos mil uno' corresponde a '2001'.
* En la columna `pais` se presentan las siguientes anomalías:
    * Datos vacíos.
    * Ciudades, e.g. _arica_.
    * Países mal escritos e.g. _shile_.
    * Países repetidos pero mal formateados, e.g. _Turkey_.
    * Cruce de información, e.g. _Iran, 2005 Qeshm earthquake_.
* En la columna `magnitud` se presentan las siguientes anomalías:
    * Datos vacíos.
    * Cruce de información, e.g. _2002-Tanzania-5.8_.
    * Valores imposibles, e.g. _9.7_.
* La columna `informacion` realmente no está entregando ninguna información valiosa al problema.

Partamos por eliminar la columna `informacion`.

In [25]:
eqk = earthquakes.drop(columns="informacion")  # A veces es importante no sobrescribir el dataframe original para realizar análisis posteriores.
eqk.head()

,año,pais,magnitud
0,2000,Turkey,6
1,2000,Turkmenistan,7
2,2000,Azerbaijan,6.5
3,2000,Azerbaijan,6.8
4,2000,Papua New Guinea,8


Respecto a la columna `año`, corregir estos errores no es difícil, pero suele ser tedioso. Aparte que si no se realiza un correcto análisis es posible no detectar estos errores a tiempo. Empecemos con los registros nulos.

In [26]:
eqk.loc[lambda x: x["año"].isnull()]

,año,pais,magnitud
225,NaN,NaN,2002-Tanzania-5.8
226,NaN,NaN,2003-japan-8.5


Veamos el archivo

In [27]:
! sed  -n "226,228p" data/earthquakes_contaminated.csv

sed: can't read data/earthquakes_contaminated.csv: No such file or directory


Toda la información está contenida en una columna!

Para editar la información usaremos dos herramientas:
 * Los métodos de `str` en `pandas`, en particular para dividir una columna.
 * `loc` para asignar los nuevos valores.

In [28]:
eqk.loc[lambda x: x["año"].isnull(), "magnitud"].str.split("-", expand=True).values

array([['2002', 'Tanzania', '5.8'],
       ['2003', 'japan', '8.5']], dtype=object)

In [29]:
eqk.loc[lambda x: x["año"].isnull(), :] = eqk.loc[lambda x: x["año"].isnull(), "magnitud"].str.split("-", expand=True).values

In [30]:
eqk.loc[[225, 226]]

,año,pais,magnitud
225,2002,Tanzania,5.8
226,2003,japan,8.5


Ahora los registros que no se pueden convertir a `numeric`. Veamos que no es posible convertirlo.

In [31]:
try:
    eqk["año"].astype(np.int)
except Exception as e:
    print(e)

invalid literal for int() with base 10: 'dos mil uno'


In [32]:
eqk["año"].str.isnumeric().fillna(False)

0      True
1      True
2      True
3      True
4      True
       ... 
223    True
224    True
225    True
226    True
227    True
Name: año, Length: 228, dtype: bool

In [33]:
eqk.loc[lambda x: ~ x["año"].str.isnumeric()]

,año,pais,magnitud
31,dos mil uno,China,5.4


Veamos el valor a cambiar

In [34]:
eqk.loc[lambda x: ~ x["año"].str.isnumeric(), "año"].iloc[0]

'dos mil uno'

Reemplazar es muy fácil!

In [35]:
eqk["año"].str.replace("dos mil uno", "2001")

0      2000
1      2000
2      2000
3      2000
4      2000
       ... 
223    1990
224    1999
225    2002
226    2003
227    2005
Name: año, Length: 228, dtype: object

Para asignar en el dataframe basta con:

In [36]:
eqk["año"] = eqk["año"].str.replace("dos mil uno", "2001").astype(np.int)

La forma encadenada sería:

In [37]:
# eqk["año"] = eqk.assign(año=lambda x: x["año"].str.replace("dos mil uno", "2001").astype("int"))

In [38]:
eqk.dtypes

año          int64
pais        object
magnitud    object
dtype: object

Finalmentem, filtremos los años necesarios:

In [39]:
eqk = eqk.query("2000 <= año <= 2011")

Siguiendo de forma análoga con la columna `magnitud`.

In [40]:
eqk.loc[lambda x: x["magnitud"].isnull()]

,año,pais,magnitud
219,2010,Colombia,NaN
220,2005,Indonesia,NaN
221,2010,Venezuela,NaN


La verdad es que no hay mucho que hacer con estos valores, por el momento no _inputaremos_ ningún valor y los descartaremos.

In [41]:
eqk = eqk.loc[lambda x: x["magnitud"].notnull()]

In [42]:
try:
    eqk["magnitud"].astype(np.float)
    print("Ya es posible transformar la columna a float.")
except:
    print("Aún no es posible transformar la columna a float.")

Ya es posible transformar la columna a float.


In [43]:
eqk = eqk.astype({"magnitud": np.float})

In [44]:
eqk.dtypes

año           int64
pais         object
magnitud    float64
dtype: object

In [45]:
eqk.magnitud.unique()

array([  6. ,   7. ,   6.5,   6.8,   8. ,   5.7,   6.4,   5.5,   6.3,
         5.4,   6.1,   6.7,   7.9,   7.2,   7.5,   5.3,   5.9,   9.7,
         5.8,   4.7,   7.6,   8.4,   5. ,   5.6,   6.6,   6.2,   7.1,
         7.3,   5.1,   5.2,   8.3,   6.9,   9.1,   4.9,   7.8,   8.6,
         7.7,   7.4,   8.5,   8.1,   8.8,   9. , -10. ])

In [46]:
eqk.query("magnitud < 0 or 9.6 < magnitud")

,año,pais,magnitud
22,2000,shile,9.7
217,2011,shile,-10.0
218,2011,shile,-10.0


In [47]:
eqk = eqk.query("0 <= magnitud <= 9.6")

In [48]:
eqk.query("magnitud < 0 or 9.6 < magnitud")

,año,pais,magnitud


Finalmente, para la columna `pais`. Comenzaremos con los nombres erróneos, estos los podemos mapear directamente.

In [49]:
map_paises = {"arica": "Chile", "shile": "Chile", "Iran, 2005 Qeshm earthquake": "Iran"}
eqk["pais"].map(map_paises).fillna(eqk["pais"])

0                Turkey
1          Turkmenistan
2            Azerbaijan
3           Azerbaijan 
4      Papua New Guinea
             ...       
215              China 
216        New Zealand 
225            Tanzania
226               japan
227               Chile
Name: pais, Length: 219, dtype: object

Para editarlo en el dataframe basta hacer un `assign`.

In [50]:
eqk = eqk.assign(pais=lambda x: x["pais"].map(map_paises).fillna(x["pais"]))

Ahora formatearemos los nombres, pasándolos a minúsculas y quitando los espacios al principio y final de cada _string_. Y ahabíamos hablado del ejemplo de _Turkey_.

In [51]:
eqk.loc[lambda x: x["pais"].apply(lambda s: "Turkey" in s), "pais"].unique()

array(['Turkey', 'Turkey '], dtype=object)

In [52]:
# Chaining method
eqk = eqk.assign(pais=lambda x: x["pais"].str.lower().str.strip())

In [53]:
eqk.loc[lambda x: x["pais"].apply(lambda s: "turkey" in s), "pais"].unique()

array(['turkey'], dtype=object)

Nota que no hay países con valores nulos porque ya fueron reparados.

In [56]:
eqk.loc[lambda x: x["pais"].isnull()]

,año,pais,magnitud


#### ¿Se puede sacar más provecho a los datos ?

No es posible crear variables nuevas o algo por el estilo, ya se hizo todo el procesamiento necesario para cumplir las reglas de negocio.

In [57]:
earthquakes.shape

(228, 4)

In [58]:
eqk.shape

(219, 3)

#### Dar respuesta

Como es un método de agregación podríamos simplemente hacer un `groupby`.

In [59]:
eqk.groupby(["pais", "año"])["magnitud"].max()

pais           año 
afghanistan    2000    6.3
               2001    5.0
               2002    7.3
               2003    5.8
               2004    6.5
                      ... 
turkmenistan   2000    7.0
united states  2001    6.8
               2003    6.6
venezuela      2006    5.5
vietnam        2005    5.3
Name: magnitud, Length: 134, dtype: float64

Sin embargo, en ocasiones, una tabla __pivoteada__ es mucho más explicativa.

In [60]:
eqk.pivot_table(
    index="pais",
    columns="año",
    values="magnitud",
    aggfunc="max",
    fill_value=""
)

año,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
pais,,,,,,,,,,,,
afghanistan,6.3,5,7.3,5.8,6.5,6.5,,,,,,
algeria,5.7,,,6.8,,,5.2,,5.5,,,
argentina,7.2,,,,6.1,,,,,,,
azerbaijan,6.8,,,,,,,,,,,
bangladesh,,,,5.6,,,,,,,,
burma,,,,,,,,,,,,6.8
chile,,6.3,,,,8,,7.7,,,8.8,
china,5.9,5.6,5.5,6.3,5.3,5.2,5,6.1,7.9,5.7,6.9,5.4
colombia,6.5,,,,,,,,5.9,,,


¿Notas las similitudes con `groupby`? Ambos son métodos de agregación, pero retornan formas de la matriz distintas.

Sin embargo, esto se vería mucho mejor con una visualización, que es lo que veremos en el próximo módulo.

In [61]:
import altair as alt
alt.themes.enable('opaque')

alt.Chart(
    eqk.groupby(["pais", "año"])["magnitud"].max().reset_index()
).mark_rect().encode(
    x='año:O',
    y='pais:N',
    color='magnitud:Q'
)

alt.Chart(...)

## Resumen

* En el _mundo real_ te encontrarás con múltiples fuentes de datos, es importante adaptarse ya que las tecnologías cambian constantemente.
* Datos deben entregar valor a través del análisis.
* Es poco probable que los datos vengan _"limpios"_.
* El análisis exploratorio de datos (EDA) es una metodología que sirve para asegurarse de la calidad de los datos.
* A medida que se tiene más experticia en el tema, mejor es el análisis de datos y por tanto, mejor son los resultados obtenidos.
* No existe un procedimiento estándar para realizar el EDA, pero siempre se debe tener claro el problema a resolver.
